In [1]:
import pandas as pd
import plotly.express as px

# Analysis

In [19]:

info = pd.read_excel('/home/leo/Code/ifpass/projet_deployment/data/get_around_delay_analysis.xlsx', sheet_name=1)
info[info['field name'] == 'checkin_type'].values

array([['checkin_type',
        "Flow used for both checkin and checkout. (ie. access and return the car)\nmobile = rental agreement signed on the owner's smartphone\nconnect = car equiped with the Connect technology , opened by the driver with his smartphone.\nNote: paper contracts were excluded from the data as we have no data on their delay at checkout and it's negligible use case"]],
      dtype=object)

In [57]:
df_delay = pd.read_excel('/home/leo/Code/ifpass/projet_deployment/data/get_around_delay_analysis.xlsx')

In [58]:
df_delay.head()

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


In [59]:
df_delay['previous_ended_rental_id'] = df_delay['previous_ended_rental_id'].astype('Int64')

In [60]:
df_merged = df_delay.merge(df_delay, left_on='rental_id', right_on='previous_ended_rental_id', suffixes=('', '_previous'))
df_merged

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes,rental_id_previous,car_id_previous,checkin_type_previous,state_previous,delay_at_checkout_in_minutes_previous,previous_ended_rental_id_previous,time_delta_with_previous_rental_in_minutes_previous
0,531158,404169,mobile,ended,29.0,<NA>,NaN,534827,404169,mobile,ended,-7.0,531158,90.0
1,533303,336902,mobile,ended,-340.0,<NA>,NaN,552606,336902,mobile,ended,-90.0,533303,600.0
2,533380,314456,connect,ended,-167.0,<NA>,NaN,542971,314456,connect,ended,-26.0,533380,690.0
3,534820,373893,connect,ended,-576.0,<NA>,NaN,543571,373893,connect,ended,-4.0,534820,150.0
4,535313,396552,mobile,ended,23.0,<NA>,NaN,513104,396552,mobile,ended,23.0,535313,720.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1836,574571,341861,connect,ended,-54.0,<NA>,NaN,571038,341861,connect,ended,-152.0,574571,540.0
1837,574596,356190,mobile,ended,10.0,<NA>,NaN,575497,356190,mobile,ended,702.0,574596,30.0
1838,567694,349618,mobile,ended,-17.0,<NA>,NaN,572870,349618,mobile,ended,-178.0,567694,210.0
1839,568465,390871,connect,ended,NaN,<NA>,NaN,566228,390871,connect,canceled,NaN,568465,60.0


In [37]:
df_delay.shape

(21310, 7)

In [5]:
df_delay.isna().sum()

rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64

In [35]:
df_delay[~df_delay['previous_ended_rental_id'].isna()]

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
6,511639,370585,connect,On time,-15.0,563782.0,570.0
19,519491,312389,mobile,delayed,58.0,545639.0,420.0
23,521156,392479,mobile,ended,NaN,537298.0,0.0
34,525044,349751,mobile,ended,NaN,510607.0,60.0
40,528808,181625,connect,On time,-76.0,557404.0,330.0
...,...,...,...,...,...,...,...
21269,568049,381499,connect,canceled,NaN,562174.0,720.0
21272,568241,396409,mobile,canceled,NaN,566136.0,570.0
21275,568523,297973,mobile,delayed,12.0,567121.0,240.0
21286,569717,377312,mobile,delayed,230.0,545045.0,90.0


In [36]:
df_delay[df_delay['rental_id'] == 563782]

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
17047,563782,370585,connect,delayed,136.0,NaN,NaN


# Preprocessing

In [6]:
def change_state(state, delay):
    if delay > 1440:
        return 'error'
    elif delay > 0:
        return 'delayed'
    elif delay < 0:
        return 'On time'
    else:
        return state

In [7]:
df_delay['state'] = df_delay.apply(lambda row: change_state(row['state'], row['delay_at_checkout_in_minutes']), axis=1)
df_delay

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,On time,-81.0,NaN,NaN
2,508131,359049,connect,delayed,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN
...,...,...,...,...,...,...,...
21305,573446,380069,mobile,ended,NaN,573429.0,300.0
21306,573790,341965,mobile,On time,-337.0,NaN,NaN
21307,573791,364890,mobile,delayed,144.0,NaN,NaN
21308,574852,362531,connect,On time,-76.0,NaN,NaN


unique car on Getaround

In [8]:
len(df_delay['car_id'].unique())

8143

Most rented car_id

In [9]:
df_delay['car_id'].value_counts().sort_values(ascending=False).head(10)

car_id
334190    33
241021    29
392479    29
342699    27
300505    24
357530    23
335276    23
368432    23
390467    22
392368    22
Name: count, dtype: int64

Number of checkin type

In [10]:
df_1 = df_delay.drop_duplicates('car_id')
df_1['checkin_type'].value_counts()

checkin_type
mobile     7467
connect     676
Name: count, dtype: int64

In [34]:
state_counts = df_delay['state'].value_counts()

state_counts_df = state_counts.reset_index()
state_counts_df.columns = ['state', 'count']

fig = px.pie(state_counts_df, names='state', values='count', title='State Distribution')
fig.update_layout(height=400, width=600, margin=dict(l=35, r=10, t=45, b=0))
fig.show()


In [33]:
df_timedelay = df_delay[df_delay['state'] == 'delayed']
df_timedelay

Q1 = df_timedelay['delay_at_checkout_in_minutes'].quantile(0.25)
Q3 = df_timedelay['delay_at_checkout_in_minutes'].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_filtered = df_timedelay[(df_timedelay['delay_at_checkout_in_minutes'] >= lower_bound) & 
                       (df_timedelay['delay_at_checkout_in_minutes'] <= upper_bound)]

fig = px.box(df_filtered, y='delay_at_checkout_in_minutes', title='Delay at Checkout in Minutes (Without Outliers)')
fig.update_yaxes(title='Minutes of delay')
fig.update_layout(height=400, width=600, margin=dict(l=35, r=10, t=45, b=0))
fig.show()

In [13]:
bin_size = 15
num_bins = int((df_timedelay['delay_at_checkout_in_minutes'].max() + bin_size) / bin_size)
fig = px.histogram(df_timedelay, x='delay_at_checkout_in_minutes', nbins=num_bins,
                   title='Distribution of Delay at Checkout in Minutes',
                   histnorm='percent')
fig.update_xaxes(range=[0, 400])
fig.show()

In [78]:
df_merged = df_delay.merge(df_delay, left_on='previous_ended_rental_id', right_on='rental_id', suffixes=('', '_previous'))

df_merged = df_merged[['rental_id', 'car_id', 'checkin_type', 'state', 'delay_at_checkout_in_minutes', 
                       'previous_ended_rental_id', 'delay_at_checkout_in_minutes_previous']]

df_merged.rename(columns={'delay_at_checkout_in_minutes': 'current_delay', 
                          'delay_at_checkout_in_minutes_previous': 'previous_delay'}, inplace=True)

df_merged


,rental_id,car_id,checkin_type,state,current_delay,previous_ended_rental_id,previous_delay
0,511639,370585,connect,ended,-15.0,563782,136.0
1,519491,312389,mobile,ended,58.0,545639,140.0
2,521156,392479,mobile,ended,NaN,537298,NaN
3,525044,349751,mobile,ended,NaN,510607,-113.0
4,528808,181625,connect,ended,-76.0,557404,-352.0
...,...,...,...,...,...,...,...
1836,568049,381499,connect,canceled,NaN,562174,73.0
1837,568241,396409,mobile,canceled,NaN,566136,16.0
1838,568523,297973,mobile,ended,12.0,567121,-275.0
1839,569717,377312,mobile,ended,230.0,545045,22.0


In [87]:
df_1 = df_merged[df_merged['previous_delay'] > 0]
df_1[df_1['state'] == 'canceled']

,rental_id,car_id,checkin_type,state,current_delay,previous_ended_rental_id,previous_delay
17,543768,374169,connect,canceled,NaN,543010,42.0
78,538906,334190,connect,canceled,NaN,556215,77.0
114,533570,271621,mobile,canceled,NaN,561375,25.0
118,536722,387431,mobile,canceled,NaN,536764,58.0
122,539151,282893,mobile,canceled,NaN,548646,201.0
...,...,...,...,...,...,...,...
1809,559133,335341,mobile,canceled,NaN,558460,13.0
1813,560787,413181,mobile,canceled,NaN,560542,4.0
1823,566008,378689,mobile,canceled,NaN,564756,39.0
1836,568049,381499,connect,canceled,NaN,562174,73.0


In [88]:

df_2 = df_merged[df_merged['previous_delay'] > 0]
df_2

,rental_id,car_id,checkin_type,state,current_delay,previous_ended_rental_id,previous_delay
0,511639,370585,connect,ended,-15.0,563782,136.0
1,519491,312389,mobile,ended,58.0,545639,140.0
5,533670,320824,connect,ended,-6.0,556563,23.0
6,534827,404169,mobile,ended,-7.0,531158,29.0
7,535611,397814,mobile,ended,68.0,535512,44.0
...,...,...,...,...,...,...,...
1829,568325,354026,mobile,ended,3.0,567791,18.0
1835,567741,294059,mobile,ended,111.0,567708,283.0
1836,568049,381499,connect,canceled,NaN,562174,73.0
1837,568241,396409,mobile,canceled,NaN,566136,16.0


In [89]:
df_merged[df_merged['previous_delay'] > 15]

,rental_id,car_id,checkin_type,state,current_delay,previous_ended_rental_id,previous_delay
0,511639,370585,connect,ended,-15.0,563782,136.0
1,519491,312389,mobile,ended,58.0,545639,140.0
5,533670,320824,connect,ended,-6.0,556563,23.0
6,534827,404169,mobile,ended,-7.0,531158,29.0
7,535611,397814,mobile,ended,68.0,535512,44.0
...,...,...,...,...,...,...,...
1829,568325,354026,mobile,ended,3.0,567791,18.0
1835,567741,294059,mobile,ended,111.0,567708,283.0
1836,568049,381499,connect,canceled,NaN,562174,73.0
1837,568241,396409,mobile,canceled,NaN,566136,16.0
